In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat,SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os


In [2]:
load_dotenv()
api_key = os.getenv('OPENROUTER_API_KEY')
model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model='nvidia/nemotron-3-nano-30b-a3b:free',
    api_key=api_key,
    model_info={
        "family": "nvidia",
        "vision":True,
        "function_calling":True,
        "json_output":False
    })

c:\Users\hardi\OneDrive\Desktop\autogen_projects\venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [3]:
founder=AssistantAgent(
    name="Founder",
    model_client=model_client,
    description="This is a founder of a startup company.",
    system_message="You are a visionary startup founder planning to revolutionize the industry with cutting-edge solutions in any field given. You provide innovative ideas and strategies to achieve success with your team that includes a CTO and a Product Manager. You debate with them to refine your startup plans. You are also responsible for the conversation and if you think if the required information is gathered, and the startup is satisfactory with the info and you approve you can end the discussion by mentioning 'APPROVE'.",
)
cto=AssistantAgent(
    name="CTO",
    model_client=model_client,
    description="This is a Chief Technology Officer of a startup company.",
    system_message="You are the CTO of a startup company, responsible for overseeing the technological direction and development of innovative solutions. You work closely with the Founder and Product Manager to ensure that the technology aligns with the company's vision and goals. You provide technical expertise, evaluate new technologies, and lead the engineering team to build scalable and efficient products.")
product_manager=AssistantAgent(
    name="ProductManager",
    model_client=model_client,
    description="This is a Product Manager of a startup company.",
    system_message="You are the Product Manager of a startup company, responsible for defining the product vision, strategy, and roadmap. You collaborate with the Founder and CTO to ensure that the product meets customer needs and market demands. You gather requirements, prioritize features, and work with cross-functional teams to deliver successful products to market.")

sharks_termination=TextMentionTermination("APPROVE")
sharks=RoundRobinGroupChat(
    participants=[founder,cto,product_manager],termination_condition=sharks_termination)
som_agent_sharks = SocietyOfMindAgent("society_of_mind", team=sharks, model_client=model_client)
summarizer=AssistantAgent(
    name="Summarizer",
    model_client=model_client,
    description="This is a Summarizer agent.",
    system_message="You are a summarizer agent. You will summarize the startup idea and its key points so that user can understand easily . Use lay man language and dont overcomplicate things. summazrize in maximum 500 words.")

startup_team=RoundRobinGroupChat(
    participants=[som_agent_sharks, summarizer],max_turns=2)

In [4]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        - MoneyJudge: A Financial Judge who evaluates the financial viability of startup ideas.
        - HumanLifeJudge: A Human Life Judge who assesses the societal impact and ethical considerations of startup ideas.
        - FutureJudge: A Future Judge who evaluates the long-term sustainability and future implications of startup ideas.
        - MarketJudge: A Market Judge who analyzes the market potential and competitive landscape of startup ideas

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)
money_judge=AssistantAgent(
    name="MoneyJudge",
    model_client=model_client,
    description="This is a Financial Judge.",
    system_message="You are a Financial Judge evaluating the financial viability and potential return on investment of startup ideas. You analyze financial projections, revenue models, and market potential to determine the economic soundness of each proposal. Your goal is to provide an objective assessment of the financial aspects of each idea.")
human_life_judge=AssistantAgent(
    name="HumanLifeJudge", 
    model_client=model_client,
    description="This is a Human Life Judge.",
    system_message="You are a Human Life Judge evaluating the societal impact and ethical considerations of startup ideas. You assess how each proposal affects human well-being, social equity, and ethical standards. Your goal is to provide an objective evaluation of the potential benefits and risks to human life associated with each idea.")
future_judge=AssistantAgent(
    name="FutureJudge", 
    model_client=model_client,
    description="This is a Future Judge.",
    system_message="You are a Future Judge evaluating the long-term sustainability and future implications of startup ideas. You consider how each proposal aligns with future trends, environmental impact, and societal evolution. Your goal is to provide an objective assessment of the potential long-term effects and sustainability of each idea.If yout debate is over with your desired conclusions you can end the discussion by mentioning 'TERMINATE'.")
market_judge=AssistantAgent(
    name="MarketJudge", 
    model_client=model_client,
    description="This is a Market Judge.",
    system_message="You are a Market Judge evaluating the market potential and competitive landscape of startup ideas. You analyze market trends, customer needs, and competitive positioning to determine the viability of each proposal in the marketplace. Your goal is to provide an objective assessment of the market opportunities and challenges associated with each idea.")
env_termination=TextMentionTermination("TERMINATE")
environmental_team=RoundRobinGroupChat(
    participants=[human_life_judge,future_judge],termination_condition=env_termination)
som_agent_judges = SocietyOfMindAgent("society_of_mind_judges", team=environmental_team, model_client=model_client)

selector_prompt = """Select an judge to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an judge from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""
termination = TextMentionTermination("TERMINATE")
judges_team = SelectorGroupChat(
    [planning_agent, money_judge, som_agent_judges, market_judge],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True, )

In [5]:
from autogen_agentchat.messages import TextMessage
async def run_team():
    task = TextMessage(content='Give a startup idea on Automation industry',source='user')
    result = await startup_team.run(task=task)
    print(result.messages[-1].content)
    print(f"Now the result goes to judges team for evaluation.")
await run_team()

Here's a clear, no-fluff summary of AdaptiveFlow – in **under 500 words**:

---

**The Problem:**  
Most automation tools (like RPA or basic AI) are *rigid*. They work only when everything goes exactly as planned. But in real life? Things break: a shipping label is missing, a payment gateway crashes, or a supplier is late. **73% of automation projects fail** because they can’t handle chaos. Fixing them means hiring engineers to rewrite code – which is slow, expensive, and frustrating.

**The Solution (AdaptiveFlow):**  
AdaptiveFlow is an **automation platform that learns from real failures and fixes itself** – without needing engineers. Think of it like a smart assistant that *grows* from mistakes.  

**How it works (simply):**  
1. **It understands your workflow** as a "living map" (e.g., "Order → Check payment → Ship label").  
2. **When something breaks** (like a missing label), it *immediately fixes it* with a new rule (e.g., "Retry sending label" or "Pause and email support").  


In [6]:
from autogen_agentchat.ui import Console
task = TextMessage(content='Give a startup idea on Coaching industry',source='user')
result = await Console(startup_team.run_stream(task=task))

---------- TextMessage (user) ----------
Give a startup idea on Coaching industry


---------- TextMessage (Founder) ----------
Here's a razor-sharp, battle-tested startup idea for the **Coaching Industry** – built on *real data*, not hype.  

---

### **Startup Name:** **GrowthPulse**  
**One-Sentence Hook:** *An AI platform that turns passive coaching into proactive client growth by predicting and fixing friction points before they cause drop-offs – without coaches spending hours on admin.*  

---

### **The Problem (Backed by Data):**  
- **70% of coaching clients quit within 90 days** (Forbes, 2023) – *not* because of bad advice, but because coaches can’t *anticipate* struggles.  
- Coaches waste **15+ hours/week** on scheduling, progress tracking, and writing reports (not coaching).  
- Existing tools (e.g., Calendly, Notion) are *reactive* – they track *after* a client drops off. **No one solves the "why" before it happens.**  

---

### **The Solution (GrowthPulse):**  
**It’s not an AI chatbot. It’s a *predictive growth engine* for coaches.**  
- **How it work

In [12]:
result.messages[-1].content

'Here’s a razor-sharp, investor-ready startup idea for the coaching industry — **under 500 words**, data-driven, and built for *real* coaching pain points (no fluff, no buzzwords):  \n\n---\n\n### **Startup Name:** **GrowthPulse**  \n**One-Sentence Hook:** *Predicts client drop-offs *before* they happen — so coaches save 12+ hours/week on admin and boost retention by 42% (no more reactive hand-holding).*  \n\n---\n\n### **The Brutal Problem (Backed by Data):**  \n- **70% of coaching clients quit within 90 days** — not because of bad advice, but because coaches *react* to problems instead of *preventing* them (Forbes, 2023).  \n- Coaches waste **15+ hours weekly** on scheduling, progress tracking, and reports — time *stolen* from actual coaching (ICF, 2023).  \n- **Current tools (Calendly, Notion) are reactive:** They only *log* drop-offs *after* clients leave. **No tool predicts or prevents churn.**  \n→ *Result:* Coaches lose clients *and* burn out.  \n\n---\n\n### **The Solution (Gro

In [13]:
judges = await Console(startup_team.run_stream(task=result.messages[-1].content))

---------- TextMessage (user) ----------
Here’s a razor-sharp, investor-ready startup idea for the coaching industry — **under 500 words**, data-driven, and built for *real* coaching pain points (no fluff, no buzzwords):  

---

### **Startup Name:** **GrowthPulse**  
**One-Sentence Hook:** *Predicts client drop-offs *before* they happen — so coaches save 12+ hours/week on admin and boost retention by 42% (no more reactive hand-holding).*  

---

### **The Brutal Problem (Backed by Data):**  
- **70% of coaching clients quit within 90 days** — not because of bad advice, but because coaches *react* to problems instead of *preventing* them (Forbes, 2023).  
- Coaches waste **15+ hours weekly** on scheduling, progress tracking, and reports — time *stolen* from actual coaching (ICF, 2023).  
- **Current tools (Calendly, Notion) are reactive:** They only *log* drop-offs *after* clients leave. **No tool predicts or prevents churn.**  
→ *Result:* Coaches lose clients *and* burn out.  

---

